In [1]:
spark

Waiting for a Spark session to start...

org.apache.spark.sql.SparkSession@ff04b95

In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.{col}

In [3]:
val spark = SparkSession.builder.appName("Spark ML").getOrCreate()

spark = org.apache.spark.sql.SparkSession@ff04b95


org.apache.spark.sql.SparkSession@ff04b95

In [5]:
var trainDf = spark
    .read
    .option("header", true)
    .option("inferSchema", true)
    .csv("lab05_train.csv")
    //.drop("_c0", "ID")

trainDf.show(1, 0, true)

-RECORD 0--------------------------------------------
 _c0                         | 333149                
 ID                          | 479990                
 CR_PROD_CNT_IL              | 0                     
 AMOUNT_RUB_CLO_PRC          | 0.0                   
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0                   
 APP_REGISTR_RGN_CODE        | null                  
 PRC_ACCEPTS_A_POS           | 0.0                   
 PRC_ACCEPTS_A_TK            | 0.0                   
 TURNOVER_DYNAMIC_IL_1M      | 0.0                   
 CNT_TRAN_AUT_TENDENCY1M     | null                  
 SUM_TRAN_AUT_TENDENCY1M     | null                  
 AMOUNT_RUB_SUP_PRC          | 0.0                   
 PRC_ACCEPTS_A_AMOBILE       | 0.0                   
 SUM_TRAN_AUT_TENDENCY3M     | null                  
 CLNT_TRUST_RELATION         | null                  
 PRC_ACCEPTS_TK              | 0.0                   
 PRC_ACCEPTS_A_MTP           | 0.0                   
 REST_DYNAMIC_FDEP_1M       

trainDf = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [6]:
val categoriesCols = List(
    "CLNT_TRUST_RELATION", 
    "APP_MARITAL_STATUS",
    "APP_KIND_OF_PROP_HABITATION", 
    "CLNT_JOB_POSITION_TYPE",
    "CLNT_JOB_POSITION", 
    "APP_DRIVING_LICENSE", 
    "APP_EDUCATION",
    "APP_TRAVEL_PASS", 
    "APP_CAR", 
    "APP_POSITION_TYPE", 
    "APP_EMP_TYPE",
    "APP_COMP_TYPE", 
    "PACK"
)

categoriesCols = List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)


List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)

In [7]:
trainDf = trainDf.drop(categoriesCols:_*).na.fill(1)

trainDf = [_c0: int, ID: int ... 102 more fields]


[_c0: int, ID: int ... 102 more fields]

In [8]:
val Array(trainDfTrain, trainDfTest) = trainDf.randomSplit(Array(0.8, 0.2))

trainDfTrain = [_c0: int, ID: int ... 102 more fields]
trainDfTest = [_c0: int, ID: int ... 102 more fields]


[_c0: int, ID: int ... 102 more fields]

In [28]:
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier, GBTClassifier}
import org.apache.spark.ml.feature.{VectorAssembler}

In [10]:
val vectorAssembler = new VectorAssembler()
    .setInputCols(trainDfTrain.drop("TARGET").columns)
    .setOutputCol("features")

vectorAssembler = vecAssembler_924ae2ed45ca


vecAssembler_924ae2ed45ca

In [11]:
val trainDfCleaned = vectorAssembler.transform(trainDfTrain).cache

trainDfCleaned = [_c0: int, ID: int ... 103 more fields]


[_c0: int, ID: int ... 103 more fields]

In [31]:
val GBTmodel = new GBTClassifier()
    .setLabelCol("TARGET")
    .setFeaturesCol("features")
    .setSeed(111)
    //.setNumTrees(100)
    .fit(trainDfCleaned)

GBTmodel = GBTClassificationModel (uid=gbtc_7546678e907f) with 20 trees


GBTClassificationModel (uid=gbtc_7546678e907f) with 20 trees

In [32]:
val predictions = GBTmodel.transform(
    vectorAssembler.transform(trainDfTest)
).cache

predictions
    .select(
        "TARGET",
        "features",
        "rawPrediction",
        "probability",
        "prediction"
    )
    .show(5)

+------+--------------------+--------------------+--------------------+----------+
|TARGET|            features|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+--------------------+----------+
|     0|(103,[0,1,3,5,9,1...|[1.12252616186674...|[0.90422290343598...|       0.0|
|     0|(103,[0,1,5,9,10,...|[1.51354176384048...|[0.95378278141033...|       0.0|
|     1|[34.0,146875.0,0....|[0.87343103926358...|[0.85155658110293...|       0.0|
|     0|[58.0,146899.0,0....|[0.89233904091790...|[0.85627355358838...|       0.0|
|     0|[60.0,146901.0,0....|[1.06759866701825...|[0.89427739989263...|       0.0|
+------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



predictions = [_c0: int, ID: int ... 106 more fields]


[_c0: int, ID: int ... 106 more fields]

In [33]:
predictions
    .select(
        "probability"
    )
    .show(1, 0, true)

-RECORD 0-----------------------------------------------
 probability | [0.9042229034359848,0.09577709656401523] 
only showing top 1 row



In [34]:
predictions.select("probability").printSchema()

root
 |-- probability: vector (nullable = true)



In [35]:
import org.apache.spark.ml.linalg.{SparseVector, Vector}
import org.apache.spark.mllib.linalg.{Vector => OldVector}

val vectorToArrayUdf = udf { vec: Any =>
    vec match {
      case v: Vector => v.toArray
      case v: OldVector => v.toArray
      case v => throw new IllegalArgumentException(
        "function vector_to_array requires a non-null input argument and input type must be " +
        "`org.apache.spark.ml.linalg.Vector` or `org.apache.spark.mllib.linalg.Vector`, " +
        s"but got ${ if (v == null) "null" else v.getClass.getName }.")
    }
  }.asNonNullable()

vectorToArrayUdf = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)


import org.apache.spark.mllib.linalg.{Vector=>OldVector}


UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)

In [36]:
predictions
    .select(vectorToArrayUdf(col("probability")).getItem(1).alias("prediction_probability"))
    .show(5)

+----------------------+
|prediction_probability|
+----------------------+
|   0.09577709656401523|
|   0.04621721858966332|
|    0.1484434188970617|
|   0.14372644641161014|
|   0.10572260010736423|
+----------------------+
only showing top 5 rows



In [37]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

In [38]:
val evaluator = new BinaryClassificationEvaluator()
    .setLabelCol("TARGET")
    .setRawPredictionCol("prediction")
    .setMetricName("areaUnderROC")

val accuracy = evaluator
    .evaluate(
        predictions.withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    )

evaluator = binEval_5ac9c57e0fef
accuracy = 0.8181375382852142


0.8181375382852142

In [39]:
val testDfData = spark
    .read
    .option("header", true)
    .option("inferSchema", true)
    .csv("/user/andrey.frolov/lab05_test.csv")
    .na.fill(1)

testDfData = [_c0: int, ID: int ... 114 more fields]


[_c0: int, ID: int ... 114 more fields]

In [40]:
val realPredictions = GBTmodel.transform(vectorAssembler.transform(testDfData)).cache

realPredictions = [_c0: int, ID: int ... 118 more fields]


[_c0: int, ID: int ... 118 more fields]

In [41]:
realPredictions
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "prediction")
    .show(5)

+------+-------------------+
|    ID|         prediction|
+------+-------------------+
|519130|0.17032485644554485|
|234045|0.04798257828110064|
|401256| 0.0481870895459362|
|551070|0.08297945813656982|
|367285| 0.0489538401801759|
+------+-------------------+
only showing top 5 rows



In [42]:
realPredictions
    .coalesce(1)
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "prediction")
    .withColumnRenamed("prediction", "target")
    .withColumnRenamed("ID", "id")
    .write
    .mode("overwrite")
    .format("csv")
    .option("header", "true")
    .option("sep", "\\t")
    .save("/user/andrey.frolov/lab05")